In [8]:
import nest_asyncio
nest_asyncio.apply()

import aiohttp
import asyncio
import json
import pandas as pd
from datetime import datetime 
from datetime import timedelta

# Define global variables for dataframes
student_df = None
engagement_df = None
ums_att_df = None
ums_avg_att_df = None
ums_upload_avg_att=None

# Edoofy app information
edoofy_base_url = "https://edoofa-portal.bubbleapps.io/api/1.1/obj"
edoofy_bearer_token = "2cde31d8f48919a2db1467cc06a56132"
edoofy_headers = {'Authorization': f'Bearer {edoofy_bearer_token}'}

# UMS app information
ums_base_url = "https://app.edoofa.com/api/1.1/obj"
ums_bearer_token = "786720e8eb68de7054d1149b56cc04f9"
ums_headers = {'Authorization': f'Bearer {ums_bearer_token}'}

# Asynchronous function to fetch data from a table
async def fetch_table_data(session, base_url, headers, table, constraints=None):
    records = []
    cursor = 0
    total_fetched = 0

    while True:
        params = {'limit': 100, 'cursor': cursor}
        if constraints:
            params['constraints'] = json.dumps(constraints)

        api_url = f"{base_url}/{table}"
        print(f"Fetching {table} data from {base_url}... Cursor: {cursor}")

        async with session.get(api_url, headers=headers, params=params) as response:
            if response.status != 200:
                print(f"Failed to fetch data from {table}: {await response.text()}")
                break

            data = await response.json()
            new_records = data['response']['results']
            records.extend(new_records)
            total_fetched += len(new_records)

            print(f"Fetched {len(new_records)} new records, Total fetched: {total_fetched}")

            cursor += 100

            if len(new_records) < 100:
                print(f"Exiting loop, fetched less than 100 records.")
                break

    df = pd.DataFrame(records)
    print(f"Fetched {len(df)} records for {table}.")
    return df

def map_students_to_engagement(student_df, engagement_df):
    # Ensure '_id' and 'EWYL-group-name' are in student_df
    if '_id' in student_df.columns and 'EWYL-group-name' in student_df.columns:
        student_id_to_EWYL = dict(zip(student_df['_id'], student_df['EWYL-group-name']))
        engagement_df['ewyl'] = engagement_df['student'].map(student_id_to_EWYL)
    else:
        print("Error: '_id' or 'EWYL-group-name' not in student_df columns")
    return engagement_df



def process_engagement_data(ums_att_df, engagement_df, student_df, ewyl_group):
    new_ums_df = pd.DataFrame(columns=['admissions-group-name', 'attendance-type', 'date',
                                       'ewyl-group-name', 'present', 'webinar'])
    # Create mappings
    student_to_kam_mapping = dict(zip(student_df['_id'], student_df['KAM-group-name']))
    student_to_ewyl_mapping = dict(zip(student_df['_id'], student_df['EWYL-group-name']))

    # Add KAM-group-name and ewyl columns to the engagement dataframe
    engagement_df['KAM-group-name'] = engagement_df['student'].map(student_to_kam_mapping)
    engagement_df['ewyl'] = engagement_df['student'].map(student_to_ewyl_mapping)

    # Find the latest date for the specific student in ums_att_df
    student_latest_ums_date = pd.to_datetime(ums_att_df[ums_att_df['ewyl-group-name'] == ewyl_group]['date'].max())

    if pd.isnull(student_latest_ums_date):
        # Process all records from engagement_df for this student
        new_ums_df['ewyl-group-name'] = engagement_df['ewyl']
        new_ums_df['admissions-group-name'] = engagement_df['KAM-group-name']
        new_ums_df['attendance-type'] = engagement_df['engagement-type']
        new_ums_df['date'] = engagement_df['engagement-date']
        new_ums_df['present'] = engagement_df['daily-attendance']
        
        print("Adding all records")
    else:
        # Process only records newer than the latest in ums_att_df
        filtered_engagement_df = engagement_df[pd.to_datetime(engagement_df['engagement-date']) > student_latest_ums_date]
        new_ums_df['ewyl-group-name'] = filtered_engagement_df['ewyl']
        new_ums_df['admissions-group-name'] = filtered_engagement_df['KAM-group-name']
        new_ums_df['attendance-type'] = filtered_engagement_df['engagement-type']
        new_ums_df['date'] = filtered_engagement_df['engagement-date']
        new_ums_df['present'] = filtered_engagement_df['daily-attendance']
        
        print("Adding new records")

    return new_ums_df
    

# Function to post processed engagement data
async def post_processed_data(session, base_url, headers, processed_data):
    api_url = f"{base_url}/Attendance"
    
    for index, row in processed_data.iterrows():
        data = {
            'admissions-group-name': row['admissions-group-name'],
            'ewyl-group-name': row['ewyl-group-name'],
            'attendance-type': row['attendance-type'],
            'present': row['present'],
            'date': row['date']
        }
        
        async with session.post(api_url, headers=headers, json=data) as response:
            if response.status == 201:
                print(f"Successfully posted data for row {index}")
            else:
                print(f"Failed to post data for row {index}: {await response.text()}")



def get_percentage(number_of_present, total_sessions):
    if total_sessions == 0:
        return 0  # Avoid division by zero
    percent = (number_of_present / total_sessions) * 100
    #print(percent)
    return percent

# Function to check if a row exists for the current month for a student
async def check_if_row_exists(student, year, month, ums_avg_att_df):
    # Filtering the ums_avg_att_df to check if there's an existing record for the student
    existing_rows = ums_avg_att_df[
        (ums_avg_att_df['ewyl-group-name'] == student) &
        (ums_avg_att_df['year'] == year) &
        (ums_avg_att_df['month'] == month)
    ]

    if not existing_rows.empty:
        # Assuming '_id' is the column name for the row ID
        return existing_rows['_id'].iloc[0]
    else:
        return False

# Function to post a new attendance summary
async def post_new_attendance_summary(session, base_url, headers, data):
    api_url = f"{base_url}/Attendance-Summary"
    async with session.post(api_url, headers=headers, json=data) as response:
        if response.status == 201:
            print("Successfully created a new row in Attendance-Summary.")
        else:
            print(f"Failed to create a new row: {await response.text()}")

# Function to patch an existing attendance summary
async def patch_attendance_summary(session, base_url, headers, row_id, data):
    api_url = f"{base_url}/Attendance-Summary/{row_id}"
    async with session.patch(api_url, headers=headers, json=data) as response:
        if response.status == 200:
            print("Successfully updated the row in Attendance-Summary.")
        else:
            print(f"Failed to update the row: {await response.text()}")

            
async def process_att_summary(session, base_url, headers, combined_df, ums_avg_att_df):
    att_summary_df = pd.DataFrame(columns=['admissions-group-name', 'attendance-percentage', 'avg-att-percent-till-last-month',
                                           'ewyl-group-name', 'first-day-of-month', 'month', 'year'])
    combined_df['date'] = pd.to_datetime(combined_df['date'])    
    unique_students = combined_df['admissions-group-name'].unique()

    for student in unique_students:
        student_df = combined_df[combined_df['admissions-group-name'] == student]

        # Get the earliest date in student_df for that student and set first_date as the 1st of that month
        first_date = pd.to_datetime(student_df['date'].min())
        first_day_current_month = first_date.replace(day=1)

        while True:
            this_month_df = student_df[(student_df['date'].dt.year == first_day_current_month.year) &
                                       (student_df['date'].dt.month == first_day_current_month.month)]

            prev_month_df = student_df[student_df['date'] < first_day_current_month]
            
            # Calculate attendance percentage for this month using get_percentage
            total_sessions_this_month = len(this_month_df)
            attended_sessions_this_month = this_month_df['present'].sum()  # Sum of True values in the 'present' column
            attendance_percentage_this_month = get_percentage(attended_sessions_this_month, total_sessions_this_month)

            
            # Calculate average attendance till last month
            if prev_month_df.empty:
                avg_att_till_last_month = 0
            else:
                total_sessions_till_last_month = len(prev_month_df)
                attended_sessions_till_last_month = prev_month_df['present'].sum()  # Sum of True values in the 'present' column
                avg_att_till_last_month = get_percentage(attended_sessions_till_last_month, total_sessions_till_last_month)
   
            new_row = {
                'admissions-group-name': student,
                'attendance-percentage': attendance_percentage_this_month,
                'avg-att-percent-till-last-month': avg_att_till_last_month,
                'ewyl-group-name': student_df['ewyl-group-name'].iloc[0] if not student_df['ewyl-group-name'].empty else 'N/A',
                'first-day-of-month': first_day_current_month.strftime("%Y-%m-%d"),
                'month': first_day_current_month.strftime("%B"),
                'year': first_day_current_month.year
            }
            att_summary_df = pd.concat([att_summary_df, pd.DataFrame([new_row])], ignore_index=True)
            
            # Calculate the first day of the next month
            next_month_first_day = (first_day_current_month + pd.offsets.MonthBegin()).normalize()
            
            # Update first_day_current_month to the first day of the next month
            first_day_current_month = next_month_first_day
            
            # Check if there is data for the next month
            if student_df[student_df['date'] >= first_day_current_month].empty:
                break              

        for index, row in att_summary_df.iterrows():
                row_id = await check_if_row_exists(row['ewyl-group-name'], row['year'], row['month'], ums_avg_att_df)

                data = {
                    'admissions-group-name': row['admissions-group-name'],
                    'attendance-percentage': row['attendance-percentage'],
                    'avg-att-percent-till-last-month': row['avg-att-percent-till-last-month'],
                    'ewyl-group-name': row['ewyl-group-name'],  
                    'first-day-of-month': row['first-day-of-month'],  
                    'month': row['month'],  
                    'year': row['year']  
                }

                # Convert any Pandas Series or non-serializable types to native Python types
                for key, value in data.items():
                    if isinstance(value, pd.Series):
                        data[key] = value.iloc[0]  # Assuming you want the first element of the Series

                if row_id:
                    await patch_attendance_summary(session, base_url, headers, row_id, data)
                    print("patch function")
                else:
                    await post_new_attendance_summary(session, base_url, headers, data)
                    print("post function")

    return att_summary_df


async def main():
    global student_df, engagement_this_month_df, engagement_previous_months_df, ums_att_df, ums_avg_att_df
    
    async with aiohttp.ClientSession() as session:
        # Fetch data from 'ums_att'
        ums_att_df = await fetch_table_data(session, ums_base_url, ums_headers, "Attendance")
        
        ums_avg_att_df = await fetch_table_data(session, ums_base_url, ums_headers, "Attendance-Summary")
        
        # Determine the latest date from 'ums_att'
        ums_latest = pd.to_datetime(ums_att_df['date'].max())
        
        # Fetch Student table from Edoofy -enter student row id
        student_constraints = [           
            {'key': 'indian-edoofian', 'constraint_type': 'equals', 'value': 'yes'}
        ]
        student_df = await fetch_table_data(session, edoofy_base_url, edoofy_headers, "Student", constraints=student_constraints)
        
        engagement_df = pd.DataFrame()
        
        for engagement_type in ['IE Call', 'IE Chat', 'Activity', 'Lesson']:
                            # Fetch engagement data for each engagement type separately
                            engagement_data = await fetch_table_data(session, edoofy_base_url, edoofy_headers, "Engagement", [{'key': 'engagement-type', 'constraint_type': 'equals', 'value': engagement_type}])
                            engagement_df = pd.concat([engagement_df, engagement_data], ignore_index=True)
                        
        engagement_df.to_csv("engagement_df.csv", index=False)
        
        all_processed_data = []

        for _, student_row in student_df.iterrows():
            ewyl_group = student_row['EWYL-group-name']
            processed_df = process_engagement_data(ums_att_df, engagement_df, student_df, ewyl_group)
            
            processed_df.to_csv("processed_df.csv", index=False)
            
            if not processed_df.empty:
                await post_processed_data(session, ums_base_url, ums_headers, processed_df)
                all_processed_data.append(processed_df)
            else:
                print(f"No data to post for student {ewyl_group}")

            # Delay between students
            print("1 second buffer before proceeding...")
            await asyncio.sleep(1)

        # After the loop, concatenate only the processed data
        if all_processed_data:
            combined_df = pd.concat(all_processed_data, ignore_index=True)
        else:
            combined_df = pd.DataFrame()
        print("New Attendance data posted in UMS... proceeding with Attendance-Summary")
        
        # Process attendance summary
        await process_att_summary(session, ums_base_url, ums_headers, combined_df, ums_avg_att_df)


await main()


Fetching Attendance data from https://app.edoofa.com/api/1.1/obj... Cursor: 0
Fetched 1 new records, Total fetched: 1
Exiting loop, fetched less than 100 records.
Fetched 1 records for Attendance.
Fetching Attendance-Summary data from https://app.edoofa.com/api/1.1/obj... Cursor: 0
Fetched 1 new records, Total fetched: 1
Exiting loop, fetched less than 100 records.
Fetched 1 records for Attendance-Summary.
Fetching Student data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 0
Fetched 100 new records, Total fetched: 100
Fetching Student data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 100
Fetched 100 new records, Total fetched: 200
Fetching Student data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 200
Fetched 100 new records, Total fetched: 300
Fetching Student data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 300
Fetched 100 new records, Total fetched: 400
Fetching Student data from https://edoofa-portal.bubblea

Fetched 100 new records, Total fetched: 1800
Fetching Engagement data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 1800
Fetched 100 new records, Total fetched: 1900
Fetching Engagement data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 1900
Fetched 100 new records, Total fetched: 2000
Fetching Engagement data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 2000
Fetched 32 new records, Total fetched: 2032
Exiting loop, fetched less than 100 records.
Fetched 2032 records for Engagement.
Fetching Engagement data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 0
Fetched 100 new records, Total fetched: 100
Fetching Engagement data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 100
Fetched 100 new records, Total fetched: 200
Fetching Engagement data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 200
Fetched 100 new records, Total fetched: 300
Fetching Engagement data from https://edoofa-por

Fetched 100 new records, Total fetched: 1500
Fetching Engagement data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 1500
Fetched 100 new records, Total fetched: 1600
Fetching Engagement data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 1600
Fetched 100 new records, Total fetched: 1700
Fetching Engagement data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 1700
Fetched 100 new records, Total fetched: 1800
Fetching Engagement data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 1800
Fetched 100 new records, Total fetched: 1900
Fetching Engagement data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 1900
Fetched 100 new records, Total fetched: 2000
Fetching Engagement data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 2000
Fetched 100 new records, Total fetched: 2100
Fetching Engagement data from https://edoofa-portal.bubbleapps.io/api/1.1/obj... Cursor: 2100
Fetched 100 new records, To

Successfully posted data for row 27
Successfully posted data for row 28
Successfully posted data for row 29
Successfully posted data for row 30
Successfully posted data for row 31
Successfully posted data for row 32
Successfully posted data for row 33
Successfully posted data for row 34
Successfully posted data for row 35
Successfully posted data for row 36
Successfully posted data for row 37
Successfully posted data for row 38
Successfully posted data for row 39
Successfully posted data for row 40
Successfully posted data for row 41
Successfully posted data for row 42
Successfully posted data for row 43
Successfully posted data for row 44
Successfully posted data for row 45
Successfully posted data for row 46
Successfully posted data for row 47
Successfully posted data for row 48
Successfully posted data for row 49
Successfully posted data for row 50
Successfully posted data for row 51
Successfully posted data for row 52
Successfully posted data for row 53
Successfully posted data for

Successfully posted data for row 251
Successfully posted data for row 252
Successfully posted data for row 253
Successfully posted data for row 254
Successfully posted data for row 255
Successfully posted data for row 256
Successfully posted data for row 257
Successfully posted data for row 258
Successfully posted data for row 259
Successfully posted data for row 260
Successfully posted data for row 261
Successfully posted data for row 262
Successfully posted data for row 263
Successfully posted data for row 264
Successfully posted data for row 265
Successfully posted data for row 266
Successfully posted data for row 267
Successfully posted data for row 268
Successfully posted data for row 269
Successfully posted data for row 270
Successfully posted data for row 271
Successfully posted data for row 272
Successfully posted data for row 273
Successfully posted data for row 274
Successfully posted data for row 275
Successfully posted data for row 276
Successfully posted data for row 277
S

...

Failed to post data for row 15236: Error: Error parsing request body: Unexpected token N in JSON at position 26

Code: 1708526120604x366461571424979700
Failed to post data for row 15237: Error: Error parsing request body: Unexpected token N in JSON at position 26

Code: 1708526121506x661897722901325700
Failed to post data for row 15238: Error: Error parsing request body: Unexpected token N in JSON at position 26

Code: 1708526122372x864517922947389800
Failed to post data for row 15239: Error: Error parsing request body: Unexpected token N in JSON at position 26

Code: 1708526123271x209484611105607900
Failed to post data for row 15240: Error: Error parsing request body: Unexpected token N in JSON at position 26

Code: 1708526124162x681328245635252400
Failed to post data for row 15241: Error: Error parsing request body: Unexpected token N in JSON at position 26

Code: 1708526125050x486249178737187000
Failed to post data for row 15242: Error: Error parsing request body: Unexpected token N

Failed to post data for row 15291: Error: Error parsing request body: Unexpected token N in JSON at position 26

Code: 1708526169839x761578336856402800
Failed to post data for row 15292: Error: Error parsing request body: Unexpected token N in JSON at position 26

Code: 1708526170815x757106810115820700
Failed to post data for row 15293: Error: Error parsing request body: Unexpected token N in JSON at position 26

Code: 1708526171709x334165115421863400
Failed to post data for row 15294: Error: Error parsing request body: Unexpected token N in JSON at position 26

Code: 1708526172630x923900188271102600
Failed to post data for row 15295: Error: Error parsing request body: Unexpected token N in JSON at position 26

Code: 1708526173511x952521444377992800
Failed to post data for row 15296: Error: Error parsing request body: Unexpected token N in JSON at position 26

Code: 1708526174398x911061907694902300
Failed to post data for row 15297: Error: Error parsing request body: Unexpected token N

Failed to post data for row 15348: Error: Error parsing request body: Unexpected token N in JSON at position 26

Code: 1708526221003x625080203237310300
Failed to post data for row 15349: Error: Error parsing request body: Unexpected token N in JSON at position 26

Code: 1708526221887x817067916938925400
Failed to post data for row 15350: Error: Error parsing request body: Unexpected token N in JSON at position 26

Code: 1708526222767x238739496513920670
Failed to post data for row 15351: Error: Error parsing request body: Unexpected token N in JSON at position 26

Code: 1708526223651x422572673752190850
Failed to post data for row 15352: Error: Error parsing request body: Unexpected token N in JSON at position 26

Code: 1708526224554x580572036222003100
Failed to post data for row 15353: Error: Error parsing request body: Unexpected token N in JSON at position 26

Code: 1708526225455x498273456343949500
Failed to post data for row 15354: Error: Error parsing request body: Unexpected token N

Failed to post data for row 15403: Error: Error parsing request body: Unexpected token N in JSON at position 26

Code: 1708526269845x563446825225573570
Failed to post data for row 15404: Error: Error parsing request body: Unexpected token N in JSON at position 26

Code: 1708526270704x261130253220535300
Failed to post data for row 15405: Error: Error parsing request body: Unexpected token N in JSON at position 26

Code: 1708526271585x727672818304623200
Failed to post data for row 15406: Error: Error parsing request body: Unexpected token N in JSON at position 26

Code: 1708526272479x918815498933345900
Failed to post data for row 15407: Error: Error parsing request body: Unexpected token N in JSON at position 26

Code: 1708526273358x985679712696703200
Failed to post data for row 15408: Error: Error parsing request body: Unexpected token N in JSON at position 26

Code: 1708526274258x512959274663900000
Failed to post data for row 15409: Error: Error parsing request body: Unexpected token N

CancelledError: 

# FINAL
